# Question-Answer System

In [1]:
import openai
import pandas as pd
from prompts import *
from langchain import OpenAI, LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.agents import ZeroShotAgent, AgentExecutor
from langchain.tools.python.tool import PythonAstREPLTool

In [2]:

def load_data(filename) -> pd.DataFrame:
    df = pd.read_csv(f"data/{filename}")

    df['Date'] = pd.to_datetime(df['Date'])
    # .dt.date
    return df



def create_pandas_dataframe_agent(
    llm,
    df: pd.DataFrame,
    prefix: str,
    suffix: str,
    format_instructions: str,
    verbose: bool,
    memory
) -> AgentExecutor:
    """Construct a pandas agent from an LLM and dataframe."""

    if not isinstance(df, pd.DataFrame):
        raise ValueError(f"Expected pandas object, got {type(df)}")

    input_variables = ["df", "input", "chat_history", "agent_scratchpad"]

    tools = [PythonAstREPLTool(locals={"df": df})]

    prompt = ZeroShotAgent.create_prompt(
        tools=tools,
        prefix=prefix,
        suffix=suffix,
        format_instructions=format_instructions,
        input_variables=input_variables
    )
    partial_prompt = prompt.partial(df=str(df.head()))

    llm_chain = LLMChain(
        llm=llm,
        prompt=partial_prompt
    )
    tool_names = [tool.name for tool in tools]

    agent = ZeroShotAgent(llm_chain=llm_chain,
                          allowed_tools=tool_names, verbose=verbose)

    return AgentExecutor.from_agent_and_tools(
        agent=agent,
        tools=tools,
        verbose=verbose,
        memory=memory
    )


In [3]:

# VARIABLES
MODEL_NAME = "gpt-4"
TEMPERATURE = 0.1
df = load_data('reidin_new.csv')
api_key = ""
org = ""

if MODEL_NAME == 'gpt-4':
    llm = ChatOpenAI(temperature=TEMPERATURE,
                     model_name=MODEL_NAME,
                     openai_api_key=api_key)
else:
    llm = OpenAI(temperature=TEMPERATURE,
                 model_name=MODEL_NAME,
                 openai_api_key=api_key)

# ViewIt OpenAI API key
openai.organization = org
openai.api_key = api_key


# Set up memory
memory = ConversationBufferMemory(memory_key="chat_history")


# AGENT CREATION HAPPENS HERE
agent = create_pandas_dataframe_agent(
    llm=llm,
    df=df,
    prefix=REIDIN_PREFIX,
    suffix=SUFFIX,
    format_instructions=FORMAT_INSTRUCTIONS,
    verbose=True,
    memory=memory
)

C:\Users\ga201\AppData\Local\Temp\ipykernel_1652\436993238.py:4: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['Date'] = pd.to_datetime(df['Date'])


In [5]:
agent.run('How many transactions are there?')



> Entering new AgentExecutor chain...
The customer is asking for the total number of transactions in the dataframe. I can use the len() function in Python to get the number of rows in the dataframe, which represents the number of transactions.
Action: python_repl_ast
Action Input: len(df)
Observation: 127134
Thought:I now know the final answer.
Final Answer: There are a total of 127,134 transactions.

> Finished chain.


'There are a total of 127,134 transactions.'